# Savannah Bikeways
- GIS File: https://data-sagis.opendata.arcgis.com/datasets/SAGIS::bikeways/about
- Plan: https://www.thempc.org/Core/Bpp#gsc.tab=0
---
This notebook is for processing and cleaning the Savannah bikeways data. This layer will be used for assigning improvements the Savannah network so that they can be assessed with BikewaySim

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import json

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

In [2]:
bikeways = gpd.read_file(Path.home()/"Documents/BikewaySim/RAW/SAGIS/Bikeways.geojson").to_crs(config['projected_crs_epsg'])
bikeways.columns

Index(['OBJECTID', 'RtA', 'RtB', 'RtC', 'SegA', 'SegB', 'SegC', 'Route_Nums',
       'Rt_Name', 'St_Name', 'Existing', 'Signed_Rt', 'Class', 'Type',
       'Status_Type', 'History', 'Avg_Daily_Traffic', 'Directional_Factor',
       'Peak_to_Daily_Factor', 'Peak_Hr_Factor', 'Peak_15_Min',
       'Total_Directional_Thru_Lanes', 'Speed_Posted', 'Speed_Effective',
       'Heavy_Veh_Percent', 'Pave_Rating_FHWA', 'Total_Width_Outlane_Shldr',
       'Occ_On_St_Park', 'Width_Pave_Outstripe_to_Edge', 'Width_On_St_Park',
       'Width_Function_of_Vol', 'St_Undivided_Unstriped',
       'Ln_Width_Avg_Effective', 'LOS_Score', 'LOS_Category', 'EXIST_CD',
       'GlobalID', 'SHAPESTLength', 'geometry'],
      dtype='object')

In [3]:
drop_cols = ['OBJECTID','RtA', 'RtB', 'RtC', 'SegA', 'SegB',
            'SegC','Route_Nums','History', 'Avg_Daily_Traffic', 'Directional_Factor',
            'Peak_to_Daily_Factor', 'Peak_Hr_Factor', 'Peak_15_Min',
            'Total_Directional_Thru_Lanes', 'Speed_Posted', 'Speed_Effective',
            'Heavy_Veh_Percent', 'Pave_Rating_FHWA', 'Total_Width_Outlane_Shldr',
            'Occ_On_St_Park', 'Width_Pave_Outstripe_to_Edge', 'Width_On_St_Park',
            'Width_Function_of_Vol', 'St_Undivided_Unstriped',
            'Ln_Width_Avg_Effective', 'LOS_Score', 'LOS_Category', 'EXIST_CD',
       'GlobalID', 'SHAPESTLength','Signed_Rt']
bikeways.drop(columns=drop_cols,inplace=True)

Existing or EXIST_CD (Existing Type) field:
- BL or 101 = Bike Lane
- CT or 102 = Cycle Track
- BP or 103 = Bike Path (i.e. Shared Use Path)
- PS or 104 = Paved Shoulder
- Narrow PS or 105 = Narrow Paved Shoulder
- SL or 106 = Shared Lane
- WCL or 107 = Wide Curb Lane
- 108 = Unopened

In [4]:
data_dictionary = {
    'BL': 'Bike Lane',
    101: 'Bike Lane',
    'CT': 'Cycle Track',
    102: 'Cycle Track',
    'BP': 'Shared Use Path',#'Bike Path (i.e. Share Use Path)',
    103: 'Shared Use Path',#'Bike Path (i.e. Share Use Path)',
    'PS': 'Paved Shoulder',
    104: 'Paved Shoulder',
    'NarrowPS': 'Narrow Paved Shoulder',
    105: 'Narrow Paved Shoulder',
    'SL': 'Shared Lane',
    106: 'Shared Lane',
    'WCL': 'Wide Curb Lane',
    107: 'Wide Curb Lane',
    108: 'Unopened'
}

#bikeways['EXIST_CD'] = bikeways['EXIST_CD'].map(data_dictionary)
bikeways['Existing'] = bikeways['Existing'].map(data_dictionary)

Status_by_Type (Status of Plan) field:
- 0 = Existing Bike Lane
- 1 = Existing Shared Use Path
- 2 = Existing Paved Shoulder
- 3 = Existing Shared Lane
- 4 = Existing Wide Curb Lane
- 5 = Recommended Bike Lane
- 6 = Recommended Shared Use Path
- 7 = Recommended Paved Shoulder
- 8 = Recommended Wide Curb Lane
- 9 = Existing Cycle Track
- 10 = Recommended Cycle Track
- 11 = Existing Narrow Paved Shoulder
- 12 = Recommended Narrow Paved Shoulder
- 13 = Recommended Shared Lane

In [5]:
data_dictionary = {
    0 : "Existing Bike Lane",
    1 : "Existing Shared Use Path",
    2 : "Existing Paved Shoulder",
    3 : "Existing Shared Lane",
    4 : "Existing Wide Curb Lane",
    5 : "Recommended Bike Lane",
    6 : "Recommended Shared Use Path",
    7 : "Recommended Paved Shoulder",
    8 : "Recommended Wide Curb Lane",
    9 : "Existing Cycle Track",
    10 : "Recommended Cycle Track",
    11 : "Existing Narrow Paved Shoulder",
    12 : "Recommended Narrow Paved Shoulder",
    13 : "Recommended Shared Lane"
}

In [6]:
bikeways['Status_Type'] = bikeways['Status_Type'].map(data_dictionary)

In [7]:
improvements = bikeways[bikeways['Status_Type'].str.contains('Recommended')].copy()
improvements.loc[:,'Recommended'] = improvements['Status_Type'].apply(lambda x: x.split('Recommended ')[-1])

#drop when not improved
drop_no_improvement = improvements['Existing'] != improvements['Recommended']
improvements = improvements[drop_no_improvement]

#drop features that aren't bicycle facilities
drop_facils = ['Shared Lane','Paved Shoulder','Narrow Paved Shoulder','Wide Curb Lane']
improvements = improvements[improvements['Recommended'].isin(drop_facils)==False]

improvements[['Existing','Recommended']].value_counts(dropna=False)

Existing        Recommended    
Shared Lane     Bike Lane          152
                Shared Use Path     91
NaN             Shared Use Path     77
Shared Lane     Cycle Track         23
Paved Shoulder  Bike Lane           18
NaN             Bike Lane           13
Wide Curb Lane  Bike Lane           12
Paved Shoulder  Shared Use Path     10
Wide Curb Lane  Cycle Track          3
NaN             Cycle Track          1
Name: count, dtype: int64

In [8]:
#convert to osm type
osm_types = ['sharrow','bike lane','buffered bike lane','cycletrack','multi use path']
savannah_conversion = {
    'Bike Lane': osm_types[1],
    'Shared Use Path': osm_types[4],
    'Cycle Track': osm_types[3]
}
improvements['savannah_osm_type'] = improvements['Recommended'].map(savannah_conversion)
improvements

,Rt_Name,St_Name,Existing,Class,Type,Status_Type,geometry,Recommended,savannah_osm_type
1,Henry/Anderson Corridor,Anderson St,Shared Lane,"CLASS II (Bike Ln, Bike Shldr)",Bike Lane,Recommended Bike Lane,"LINESTRING (994728.459 749491.076, 994919.124 ...",Bike Lane,bike lane
2,Johnny Mercer Corridor,Johnny Mercer Blvd,Shared Lane,"CLASS II (Bike Ln, Bike Shldr)",Bike Lane,Recommended Bike Lane,"LINESTRING (1014680.380 742273.428, 1013972.05...",Bike Lane,bike lane
4,Skidaway Rd Corridor,Skidaway Rd,Shared Lane,"CLASS I (Bike Path, Bike/Ped Sidewalk)",Bike Path,Recommended Shared Use Path,"MULTILINESTRING ((994374.743 725157.691, 99453...",Shared Use Path,multi use path
5,Victory Sq Cross Connectors,Sunset Blvd,Shared Lane,"CLASS II (Bike Ln, Bike Shldr)",Bike Lane,Recommended Bike Lane,"LINESTRING (996645.970 741570.273, 998533.876 ...",Bike Lane,bike lane
6,Pooler Central Corridor,None,NaN,"CLASS I (Bike Path, Bike/Ped Sidewalk)",Bike Path,Recommended Shared Use Path,"LINESTRING (940149.723 774322.738, 940145.725 ...",Shared Use Path,multi use path
...,...,...,...,...,...,...,...,...,...
855,Southwest Sector Bikeways,Future development,NaN,"CLASS II (Bike Ln, Bike Shldr)",Bike Lane,Recommended Bike Lane,"LINESTRING (919353.904 739881.128, 919436.237 ...",Bike Lane,bike lane
857,Tybee Island Bikeways,None,NaN,"CLASS I (Bike Path, Bike/Ped Sidewalk)",Bike Path,Recommended Shared Use Path,"LINESTRING (1060444.166 736728.940, 1060409.97...",Shared Use Path,multi use path
858,MTTS/TG/SRR State Rts,Liberty St,Shared Lane,"CLASS II (Bike Ln, Bike Shldr)",Bike Lane,Recommended Bike Lane,"LINESTRING (988543.927 756246.149, 988185.145 ...",Bike Lane,bike lane
864,Skidaway Rd Corridor,Skidaway Rd,Shared Lane,"CLASS I (Bike Path, Bike/Ped Sidewalk)",Bike Path,Recommended Shared Use Path,"LINESTRING (994533.605 724793.178, 994546.663 ...",Shared Use Path,multi use path


In [9]:
improvements.columns

Index(['Rt_Name', 'St_Name', 'Existing', 'Class', 'Type', 'Status_Type',
       'geometry', 'Recommended', 'savannah_osm_type'],
      dtype='object')

In [10]:
improvements.rename(columns={
    'Rt_Name':'savannah_id',
    'St_Name':'savannah_name',
},inplace=True)
improvements = improvements[['savannah_id','savannah_name','savannah_osm_type','geometry']]

It looks like the features with null street names are new mups that wouldn't be connected, so remove until the script for adding them is available

In [11]:
improvements[improvements['savannah_name'].isna()].explore()

In [12]:
improvements = improvements[improvements['savannah_name'].notna()]

In [13]:
improvements['savannah_id'].nunique()

65

In [14]:
improvements

,savannah_id,savannah_name,savannah_osm_type,geometry
1,Henry/Anderson Corridor,Anderson St,bike lane,"LINESTRING (994728.459 749491.076, 994919.124 ..."
2,Johnny Mercer Corridor,Johnny Mercer Blvd,bike lane,"LINESTRING (1014680.380 742273.428, 1013972.05..."
4,Skidaway Rd Corridor,Skidaway Rd,multi use path,"MULTILINESTRING ((994374.743 725157.691, 99453..."
5,Victory Sq Cross Connectors,Sunset Blvd,bike lane,"LINESTRING (996645.970 741570.273, 998533.876 ..."
8,Savannah-Whitemarsh Corridor,Islands Expressway,bike lane,"LINESTRING (1008031.198 752053.396, 1008390.34..."
...,...,...,...,...
854,Truman Greenway,,multi use path,"LINESTRING (990474.910 729602.845, 990519.083 ..."
855,Southwest Sector Bikeways,Future development,bike lane,"LINESTRING (919353.904 739881.128, 919436.237 ..."
858,MTTS/TG/SRR State Rts,Liberty St,bike lane,"LINESTRING (988543.927 756246.149, 988185.145 ..."
864,Skidaway Rd Corridor,Skidaway Rd,multi use path,"LINESTRING (994533.605 724793.178, 994546.663 ..."


In [15]:
#reconcile with osm

# Export

In [ ]:
improvements.to_file(config['bicycle_facilities_fp']/'network_modifications.gpkg',layer='savannah')

In [ ]:
m = improvements.explore('savannah_osm_type',tiles='CartoDBpositron')
m

In [ ]:
m.save(config['bicycle_facilities_fp']/'savannah_improvements.html')